In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 17: Two Steps Forward ---</h2><p>You're trying to access a secure vault protected by a <code>4x4</code> grid of small rooms connected by doors. You start in the top-left room (marked <code>S</code>), and you can access the vault (marked <code>V</code>) once you reach the bottom-right room:</p>
<pre><code>#########
#S| | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | |  
####### V
</code></pre>
<p>Fixed walls are marked with <code>#</code>, and doors are marked with <code>-</code> or <code>|</code>.</p>
<p>The doors in your <em>current room</em> are either open or closed (and locked) based on the hexadecimal <a href="https://en.wikipedia.org/wiki/MD5">MD5</a> hash of a passcode (your puzzle input) followed by a sequence of uppercase characters representing the <em>path you have taken so far</em> (<code>U</code> for up, <code>D</code> for down, <code>L</code> for left, and <code>R</code> for right).</p>
<p>Only the first four characters of the hash are used; they represent, respectively, the doors <em>up, down, left, and right</em> from your current position. Any <code>b</code>, <code>c</code>, <code>d</code>, <code>e</code>, or <code>f</code> means that the corresponding door is <em>open</em>; any other character (any number or <code>a</code>) means that the corresponding door is <em>closed and locked</em>.</p>
<p>To access the vault, all you need to do is reach the bottom-right room; reaching this room opens the vault and all doors in the maze.</p>
<p>For example, suppose the passcode is <code>hijkl</code>. Initially, you have taken no steps, and so your path is empty: you simply find the MD5 hash of <code>hijkl</code> alone. The first four characters of this hash are <code>ced9</code>, which indicate that up is open (<code>c</code>), down is open (<code>e</code>), left is open (<code>d</code>), and right is closed and locked (<code>9</code>). Because you start in the top-left corner, there are no "up" or "left" doors to be open, so your only choice is <em>down</em>.</p>
<p>Next, having gone only one step (down, or <code>D</code>), you find the hash of <code>hijkl<em>D</em></code>. This produces <code>f2bc</code>, which indicates that you can go back up, left (but that's a wall), or right. Going right means hashing <code>hijkl<em>DR</em></code> to get <code>5745</code> - all doors closed and locked. However, going <em>up</em> instead is worthwhile: even though it returns you to the room you started in, your path would then be <code>DU</code>, opening a <em>different set of doors</em>.</p>
<p>After going <code>DU</code> (and then hashing <code>hijkl<em>DU</em></code> to get <code>528e</code>), only the right door is open; after going <code>DUR</code>, all doors lock. (Fortunately, your actual passcode is <span title="It took four days to rescue the engineer that tried this.">not <code>hijkl</code></span>).</p>
<p>Passcodes actually used by Easter Bunny Vault Security do allow access to the vault if you know the right path.  For example:</p>
<ul>
<li>If your passcode were <code>ihgpwlah</code>, the shortest path would be <code>DDRRRD</code>.</li>
<li>With <code>kglvqrro</code>, the shortest path would be <code>DDUDRLRRUDRD</code>.</li>
<li>With <code>ulqzkmiv</code>, the shortest would be <code>DRURDRUDDLLDLUURRDULRLDUUDDDRR</code>.</li>
</ul>
<p>Given your vault's passcode, <em>what is the shortest path</em> (the actual path, not just the length) to reach the vault?</p>
</article>


In [2]:
from collections import deque
from hashlib import md5


def shortest_path(key: str) -> str:
    queue = deque([(0, 0, "")])

    while queue:
        r, c, path = queue.popleft()
        if r == 3 and c == 3:
            return path

        doors = md5(f"{key}{path}".encode()).hexdigest()[:4]
        if r - 1 >= 0 and doors[0] in "bcdef":
            queue.append((r - 1, c, path + "U"))

        if r + 1 < 4 and doors[1] in "bcdef":
            queue.append((r + 1, c, path + "D"))

        if c - 1 >= 0 and doors[2] in "bcdef":
            queue.append((r, c - 1, path + "L"))

        if c + 1 < 4 and doors[3] in "bcdef":
            queue.append((r, c + 1, path + "R"))

    return ""


tests_shortest_path = [
    {
        "name": "Example 1",
        "key": "ihgpwlah",
        "expected": "DDRRRD",
    },
    {
        "name": "Example 2",
        "key": "kglvqrro",
        "expected": "DDUDRLRRUDRD",
    },
    {
        "name": "Example 3",
        "key": "ulqzkmiv",
        "expected": "DRURDRUDDLLDLUURRDULRLDUUDDDRR",
    },
    {
        "name": "Example 4",
        "key": "hijkl",
        "expected": "",
    },
]

run_tests_params(shortest_path, tests_shortest_path)


Test Example 1 passed, for shortest_path.
Test Example 2 passed, for shortest_path.
Test Example 3 passed, for shortest_path.
Test Example 4 passed, for shortest_path.
Success


In [3]:
shortest_path("njfxhljp")

'DURLDRRDRD'

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>DURLDRRDRD</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You're curious how robust this security solution really is, and so you decide to find longer and longer paths which still provide access to the vault. You remember that paths always end the first time they reach the bottom-right room (that is, they can never pass through it, only end in it).</p>
<p>For example:</p>
<ul>
<li>If your passcode were <code>ihgpwlah</code>, the longest path would take <code>370</code> steps.</li>
<li>With <code>kglvqrro</code>, the longest path would be <code>492</code> steps long.</li>
<li>With <code>ulqzkmiv</code>, the longest path would be <code>830</code> steps long.</li>
</ul>
<p></p>
<p>What is the <em>length of the longest path</em> that reaches the vault?</p>
</article>

</main>


In [4]:
def longest_path(key: str) -> int:
    def dfs(r: int, c: int, path: str) -> int:
        if r == 3 and c == 3:
            return len(path)

        maximum = 0
        doors = md5(f"{key}{path}".encode()).hexdigest()[:4]

        if r - 1 >= 0 and doors[0] in "bcdef":
            maximum = max(maximum, dfs(r - 1, c, path + "U"))

        if r + 1 < 4 and doors[1] in "bcdef":
            maximum = max(maximum, dfs(r + 1, c, path + "D"))

        if c - 1 >= 0 and doors[2] in "bcdef":
            maximum = max(maximum, dfs(r, c - 1, path + "L"))

        if c + 1 < 4 and doors[3] in "bcdef":
            maximum = max(maximum, dfs(r, c + 1, path + "R"))

        return maximum

    return dfs(0, 0, "")


tests_longest_path = [
    {
        "name": "Example 1",
        "key": "ihgpwlah",
        "expected": 370,
    },
    {
        "name": "Example 2",
        "key": "kglvqrro",
        "expected": 492,
    },
    {
        "name": "Example 3",
        "key": "ulqzkmiv",
        "expected": 830,
    },
    {
        "name": "Example 4",
        "key": "hijkl",
        "expected": 0,
    },
]

run_tests_params(longest_path, tests_longest_path)


Test Example 1 passed, for longest_path.
Test Example 2 passed, for longest_path.
Test Example 3 passed, for longest_path.
Test Example 4 passed, for longest_path.
Success


In [5]:
longest_path("njfxhljp")

650

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>650</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2016">return to your Advent calendar</a> and try another puzzle.</p>

</main>
